# Michelin Restaurant Dataset: Preparation

The dataset comes from [Kaggle](https://www.kaggle.com/datasets/ngshiheng/michelin-guide-restaurants-2021)

In [0]:
import pandas as pd
import os
from pyspark import SparkFiles
from pyspark.sql.functions import *

### Catalog, Schema Creation

In [0]:
catalog_ = os.getenv('CATALOG_NAME')
schema_ = os.getenv('SCHEMA_NAME')
data_path_ = os.getenv('DATA_PATH')

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS "+catalog_)
spark.sql("CREATE SCHEMA IF NOT EXISTS "+catalog_+"."+schema_)

In [0]:
spark.sql("USE CATALOG "+catalog_)
spark.sql("USE SCHEMA "+schema_)

### Data Ingestion from Volume [Bronze]

There is a problem in the csv file, especially on "", """ or """" followed by a comma within string descriptions, which results in a string trucation.

There are however empty fields denoted by ,"", (in one instance this is followed by a new line character instead of a comma, i.e. restaurant name Goosefoot), so a direct replacement cannot work.

The idea is to perform these steps:
- Replace ,"", with < (non existent char in the whole file)
- Replace """" with ' as they are all inside quotated fields ("""", doesn't exist)
- Replace """(+new line) with "(+new line)
- Replace ,""" with ,"
- Replace """, with '", (7 instances)
- Handle remaining """ manually (2 instances)
- Replace "" with '
- Restore, by replacing < with ,"", (handle the one instance above manually)
- Handle Goosefoot restaurant line separator manually

After that, the descriptions should be cleaned up and the shortest one is 36 chars, which is also in the original file.

In [0]:
# Download the file to a volume
import urllib

"""
volume_ = catalog_+'.'+schema_+'.init'
spark.sql("CREATE VOLUME IF NOT EXISTS "+volume_)
urllib.request.urlretrieve(
  data_path_,
  "/Volumes/"+catalog_+"/"+schema_+"/init/michelin_data.csv")
"""
filepath_ = f"dbfs:/Volumes/"+catalog_+"/"+schema_+"/init/michelin_data.csv"

In [0]:
bronze_ = (spark.
           read.
           option("header", True).
           option("delimiter", ",").
           option("encoding", "utf-8").
           option("quotechar", '"').
           option("doublequote", False).
           format("csv").
           load(filepath_).
           createOrReplaceTempView("bronze_v"))

display(spark.sql("SELECT * FROM bronze_v LIMIT 10;"))


In [0]:
%sql
DROP TABLE IF EXISTS bronze_data;
CREATE TABLE bronze_data AS
  select * from bronze_v;

### Data Exploration

In [0]:
bronze_df = spark.sql("SELECT * FROM bronze_data")
display(dbutils.data.summarize(bronze_df))

In [0]:
%sql
-- -- -- Price
select Price, len(Price)::int price_score, count(*) from bronze_data group by 1,2 order by 1,2;

In [0]:
%sql
select * from bronze_data where Price is null;

In [0]:
%sql
-- -- -- Award
select Award, 
  case Award
  when '1 Star' then 1
  when '2 Stars' then 2
  when '3 Stars' then 3
  when 'Bib Gourmand' then 0.5
  when 'Selected Restaurants' then 0.25
  end as Stars_score
  ,count(*)
  from bronze_data group by 1,2 order by 1,2;


In [0]:
%sql
-- -- -- Quality Ratio
with d0 as (
  select *
    ,case Award
      when '1 Star' then 1
      when '2 Stars' then 2
      when '3 Stars' then 3
      when 'Bib Gourmand' then 0.5
      when 'Selected Restaurants' then 0.25
      end as Stars_score
  from bronze_data
)
select 
  Stars_score
  ,len(Price)::float Price_score
  ,Stars_score / len(Price)::float as Quality
  , count(*) recs
from d0
where Price is not null
group by 1,2,3 order by 1,2,3;

In [0]:
%sql
-- -- -- Cuisine
select cuisine, count(*) from bronze_data group by 1 order by 1;

In [0]:
%sql
-- -- -- Split the cuisine as some values are comma separated
with d0 as (
  select
    trim(case when cuisine not like '%,%' then cuisine else split_part(cuisine, ',', 1) end) as Cuisine_l1,
    trim(case when cuisine not like '%,%' then cuisine else split_part(cuisine, ',', 2) end) Cuisine_l2,
    name
    from bronze_data
)
    select Cuisine_l1, count(*) recs
    from d0
    group by 1 order by 2 desc;

In [0]:
%sql
select * from bronze_data where cuisine = "'";

In [0]:
%sql
-- -- -- Location
select location,
  trim(case when location not like '%,%' then location else split_part(location, ',', 1) end) as City,
  trim(case when location not like '%,%' then location else split_part(location, ',', 2) end) Country,
 count(*) as recs_
from bronze_data
group by 1,2,3 order by 3,1;

In [0]:
%sql
-- -- -- Country
with d0 as (
  select *
  ,trim(case when location not like '%,%' then location else split_part(location, ',', 1) end) as City
  ,trim(case when location not like '%,%' then location else split_part(location, ',', 2) end) as Country
  from bronze_data
)
  select country, count(*) recs_
  from d0
  group by 1 order by 2 desc;

**`Note about Countries`**: To be recognized by maps, we should make sure these adhere to the ISO3 standard country names.

The names can be found from the gapminder dataset:

In [0]:
# Iso3 Countries
import plotly.express as px
df = px.data.gapminder()
iso3_codes = df['iso_alpha'].unique()
country_names = df['country'].unique()

# Current countries
countries__ = spark.sql("""
                        with d0 as (
                            select *
                            ,trim(case when location not like '%,%' then location else split_part(location, ',', 1) end) as City
                            ,trim(case when location not like '%,%' then location else split_part(location, ',', 2) end) as Country
                            from bronze_data
                          )
                            select distinct
                              case when country = 'Hong Kong' or country = 'Hong Kong SAR China' then 'Hong Kong, China'
                                when country = 'Czechia' then 'Czech Republic'
                                when country = 'Türkiye' then 'Turkey'
                                when country = 'USA' then 'United States'
                                when country = 'China Mainland' then 'China'
                                when country = 'Dubai' or country = 'Abu Dhabi' then 'United Arab Emirates'
                                else country end as country
                            from d0;
                        """).toPandas()
countries_ = countries__['country'].to_list()

countries_to_fix = [c for c in countries_ if c not in country_names]
print(":: Here a list of countries that wouldn't get recognized:")
print(countries_to_fix)

print(country_names)

In [0]:
%sql
-- -- -- Description
select len(Description), count(*) from bronze_data group by 1 order by 1;

In [0]:
%sql
select name, len(description), description from bronze_data where len(description) < 100
order by 2 asc; -- There are some descriptions that include nested "", therefore they get imported in a wrong way.

In [0]:
%sql
select * from bronze_data where Name = 'Heng Gi Goose and Duck Rice';

### Data Cleaning [Silver]

In [0]:
%sql
DROP TABLE IF EXISTS silver_data;
CREATE TABLE silver_data AS
with d0 as (
  select 
    concat('res-id-', cast(100000+row_number() over(order by Longitude::float, Latitude::float, `Location`, `Name`) as string)) Res_ID,
    `Name`, Address, `Location`,
    trim(case when location not like '%,%' then `Location` else split_part(location, ',', 1) end) as City,
    trim(case when location not like '%,%' then `Location` else split_part(location, ',', 2) end) Country_,
    Longitude::float, Latitude::float, PhoneNumber, `Url` as MichelineUrl, WebsiteUrl,
    Cuisine, 
    trim(case when cuisine not like '%,%' then cuisine else split_part(cuisine, ',', 1) end) as Cuisine_l1,
    trim(case when cuisine not like '%,%' then cuisine else split_part(cuisine, ',', 2) end) Cuisine_l2,
    Price, len(Price)::int Price_score,
    Award,
    case Award
      when '1 Star' then 1
      when '2 Stars' then 2
      when '3 Stars' then 3
      when 'Bib Gourmand' then 0.5
      when 'Selected Restaurants' then 0.25
      end as Stars_score,
    GreenStar::int,
    FacilitiesAndServices, Description
  from bronze_data
  where 
    Price is not null -- Removing one Japanese restaurant
), d1 as (
  select *,
    case when Country_ = 'Hong Kong' or Country_ = 'Hong Kong SAR China' then 'Hong Kong, China'
        when Country_ = 'Czechia' then 'Czech Republic'
        when Country_ = 'Türkiye' then 'Turkey'
        when Country_ = 'USA' then 'United States'
        when Country_ = 'China Mainland' then 'China'
        when Country_ = 'Dubai' or Country_ = 'Abu Dhabi' then 'United Arab Emirates'
        else Country_ end
    as Country,
    Stars_score / Price_score::float as Quality_ratio
    from d0
)
select * EXCEPT(Country_)
from d1;

In [0]:
silver_df = spark.sql("SELECT * FROM silver_data")
display(silver_df)

In [0]:
## Export Silver Data to Parquet
vol_ = "/Volumes/michelin_ga/restaurants/init/silver_data.parquet"
(silver_df
      .write
      .format("parquet")
      .mode("overwrite")
      .save(vol_))